In [ ]:
!wget https://objectstore.e2enetworks.net/ai4b-public-nlu-nlg/indic-corp-frozen-for-the-paper-oct-2022/mr.txt

In [1]:
# Load the dataset

k = 50000
input_file_path = './data/mr.txt'
output_file_path = f"./data/mr_{k}.txt"

# Function to read the first k lines from the input file and write them to the output file
def read_and_write_first_k_lines(input_file, output_file, num_lines=1000):
    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            for i in range(num_lines):
                line = infile.readline()
                if not line:  # End of file reached before 1000 lines
                    break
                outfile.write(line)
        print(f"Successfully wrote the first {num_lines} lines to {output_file}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function
read_and_write_first_k_lines(input_file_path, output_file_path, k)

data_file = output_file_path
with open(data_file, 'r') as file:
    lines = file.readlines()

Successfully wrote the first 50000 lines to ./data/mr_50000.txt.


In [17]:
import re
import string

def remove_punctuations(word):
    return word.translate(str.maketrans('', '', string.punctuation))

# Create consecutive pairs
all_words = []
for line in lines:
    if line.strip() != "":
        words = line.split(" ")
        # Remove blank words
        words = [remove_punctuations(word.strip()) for word in words if word.strip() != "" ]
        all_words.extend(words)

all_words  = ['sos'] + all_words + ['eos']
print(len(all_words))
print(len(list(set(all_words))))

1041670
140310


In [11]:
all_words = []
_ = [all_words.extend([word1, word2]) for word1, word2 in word_pairs]
distinct_words = list(set(all_words))

word_to_i = {word: i for i, word in enumerate(distinct_words)}
i_to_word = {i: word for word, i in word_to_i.items()}
print(len(distinct_words))

140310


In [12]:
# The Number of words is too damn high. Let's filter out the ones which occurs only once

word_pairs_indexed = [(word_to_i[word1], word_to_i[word2]) for word1, word2 in word_pairs]
high_freq_pairs = [x for x, freq in freq_dict.items() if freq > 20]

# Hack to select from large word pairs, store sum of their indexes. This will give some extra, but that's okay
high_freq_pairs_key = [word_to_i[word1] + word_to_i[word2] for word1, word2 in high_freq_pairs]
high_freq_pairs_key = set(high_freq_pairs_key)

freq_word_pairs = []
for word1, word2 in word_pairs:
    if (word_to_i[word1] + word_to_i[word2]) in high_freq_pairs_key:
        freq_word_pairs.extend([(word1, word2)])
all_words = []
_ = [all_words.extend([word1, word2]) for word1, word2 in freq_word_pairs]
distinct_words = list(set(all_words))
print(f"{len(all_words)=}")
print(f"{len(distinct_words)=}")

len(all_words)=288716
len(distinct_words)=9180


In [13]:
word_to_i = {word: i for i, word in enumerate(distinct_words)}
i_to_word = {i: word for word, i in word_to_i.items()}

In [ ]:
context = 3
for line in lines[:10]:
    if line.strip() != "":
        words = line.split(" ")
        # Remove blank words
        words = [word.strip() for word in words if word.strip() != "" ]
        if len(words) <= context:
            # Skip since too few words in the sentence
            break
        all_words.extend(words)
        # Start sos and eos character
        words_augmented  = ['sos'] + words + ['eos']
        for word1, word2 in zip(words_augmented, words_augmented[1:]):
